In [ ]:
import flavio
import flavio.plots as fpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
obs = '<P5p>(B0->K*mumu)';

In [ ]:
flavio.Observable[obs].get_measurements()

In [ ]:
include_measurements=[
    'LHCb B->K*mumu 2020 P 0.1-0.98',
    'LHCb B->K*mumu 2020 P 1.1-2.5',
    'LHCb B->K*mumu 2020 P 2.5-4',
    'LHCb B->K*mumu 2020 P 4-6',
    'LHCb B->K*mumu 2020 P 15-19',
]

In [ ]:
bin_list=[(0.1,0.98), (1.1,2.5), (2.5,4), (4,6), (15,19)]

In [ ]:
fpl.bin_plot_exp(obs, include_measurements=include_measurements, col_dict={'LHCb': 'black'})

fpl.bin_plot_th(obs, bin_list=bin_list, N=50, threads=8, label='SM')

plt.legend()
plt.ylim(-1,1)
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);

In [ ]:
from wilson import Wilson

w = Wilson({'C9_bsmumu':-1}, 4.8, 'WET', 'flavio')

In [ ]:
fpl.bin_plot_exp(obs, include_measurements=include_measurements, col_dict={'LHCb': 'black'})

fpl.bin_plot_th(obs, bin_list=bin_list, N=50, threads=8, label='SM')

fpl.bin_plot_th(obs, wc=w, bin_list=bin_list, N=50, threads=8, label='$C_9=-1$', fc='C2')

plt.legend()
plt.ylim(-1,1)
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);